<a href="https://colab.research.google.com/github/whispermy/novatus_academia/blob/main/Week4_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AI 노바투스과정**

Week 4: 딥러닝 기초 II (2021-08-27)

실습 #3 [**심화 실습**] : 순환 신경망 학습 실습하기 (텍스트 데이터 생성하기)

In [ ]:
### 필요 패키지 불러오기

import numpy as np
import tensorflow.keras as keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## 1. 데이터셋 준비

In [ ]:
# 데이터셋 받아와서 charcter 의 모음인 string 으로 저장하기

shakespeare_url = "https://homl.info/shakespeare" 
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [ ]:
# 데이터 살피기

print(shakespeare_text[:1000])

In [ ]:
# 각 글자를 정수로 코딩하기

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])
tokenizer.texts_to_sequences(["First"])

In [ ]:
# tokenizer 사용해보기

tokenizer.sequence_to_texts([[20,6,9,8,3]])

In [ ]:
# 서로 다른 글자의 개수

max_id=len(tokenizer.word_index)

In [ ]:
# 전체 글자의 개수 

dataset_size=tokenizer.document_count

In [ ]:
tokenizer.word_index

In [ ]:
#학습용 데이터셋 분리하기

train_size = dataset_size * 40 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
# window() 함수 이용하여 하나의 긴 글을 길이가 101 인 여러 구간으로 분리

n_steps = 100
window_length = n_steps + 1 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
# 구간들을 셔플 (shuffle) 한 후 batch 로 나누고 첫 100 개 글자와 뒤 100 개 글자를 분리

batch_size = 10000
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))



In [ ]:
# 각 정수 값을 one-hot 벡터로 바꾸기

dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1) # 데이터 prefetching

## 2. GRU 모형 만들고 학습하기

In [ ]:
# gpu 설정
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
dropout=0.2, recurrent_dropout=0))
model.add(keras.layers.GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

##3. 글자 생성하기

In [ ]:
# 입력 데이터를 숫자 & one-hot vector 로 변환하는 함수 만들기

def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts)) - 1
  return tf.one_hot(X, max_id)

In [ ]:
import numpy as np
# "How are yo" 예측
X_new = preprocess(["How are yo"])
y_proba=model.predict(X_new)
Y_pred=y_proba.argmax(axis=2)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

In [ ]:
# "hel" 예측

X_new = preprocess(["hel"])
y_proba=model.predict(X_new)
Y_pred=y_proba.argmax(axis=2)
print(Y_pred)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)[0]

In [ ]:
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

## 4. 가짜 글 생성하기

In [ ]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [ ]:
print(complete_text("t", temperature=0.2))


In [ ]:
print(complete_text("w", temperature=2))


## 5. Stateful RNN 모형으로 해보기

In [ ]:
## 학습용 데이터 준비

# ***주의*** 각 입력 시퀀스는 이전 시퀀스와 이어져야 하므로,  
# shift=n_steps 으로 하며, shuffle() 을 사용하지 말것.


dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# 모형의 골격 만들기 (Stateful=True 로 설정)

batch_size = 1
model = keras.models.Sequential()
model.add( keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0,
batch_input_shape=[batch_size, None, max_id]) )
model.add( keras.layers.GRU(128, return_sequences=True, stateful=True,
dropout=0.2, recurrent_dropout=0) )
model.add( keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")))


In [ ]:
# callback 만들기 (다음 epoch 으로 넘어가기 전에 state 재설정하는 callback)

class ResetStatesCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs):
    self.model.reset_states()

In [ ]:
# 모형 컴파일 및 학습하기

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset, epochs=20, callbacks=[ResetStatesCallback()])